In [47]:
import networkx as nx
import matplotlib.pyplot as plt
import random
from pcc_implementacao import run_simulation, visualize_communities, get_dict_nodes_owner, check_cluster_label, check_average_node_potential
import pandas as pd
import plotly.express as px
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Particle competition for complex network community detection
### Summary of the Process  
1. Initialization: Start with several particles placed randomly in the network, each with low potential.  
2. Iteration: In each step, particles choose a neighboring node to visit based in a mix of random and deterministic rules.  
3. Update Ownership: Depending on the visit outcome, update the ownership and potential of both the particle and the node.  
4. Convergence: Repeat the process until each community is owned by a single particle, indicating successful community detection.  


# Inicialização

2. Criação de uma rede aleatória clusterizada

In [ ]:
G_test = nx.connected_caveman_graph(3, 5)
nx.draw(G_test)

In [ ]:
p_det = 0.6  # Probability of deterministic movement
#iterations = 9000 # Number of iterations
M = 3
G_result, parts = run_simulation(G_test, M)
visualize_communities(G_result)

In [ ]:
G_test = nx.connected_caveman_graph(6, 20)
nx.draw(G_test)

In [ ]:
p_det = 0.6  # Probability of deterministic movement
iterations = 9000 # Number of iterations
M = 6
G_sim, parts = run_simulation(G, M, iterations)
visualize_communities(G_sim)

# Dataset

In [6]:
df_sar = pd.read_csv('data/sar_dataset.csv')
df_sar

,feature_1,feature_2,true_label,observed_label
0,1.496714,0.861736,positive,0
1,1.647689,2.523030,positive,0
2,0.765847,0.765863,positive,0
3,2.579213,1.767435,positive,1
4,0.530526,1.542560,positive,0
5,0.536582,0.534270,positive,0
6,1.241962,-0.913280,positive,0
7,-0.724918,0.437712,positive,0
8,-0.012831,1.314247,positive,0
9,0.091976,-0.412304,positive,0


In [7]:
px.scatter(df_sar, x = 'feature_1', y = 'feature_2', color = 'true_label')

In [8]:
# --- 1. Define Nodes ---
# Nodes will correspond to the index of the dataframe rows
nodes = df_sar.index

# --- 2. Calculate Neighbors ---
# Select the features to calculate distance
features = df_sar[['feature_1', 'feature_2']].values

# Choose the number of neighbors (k)
k = 3

# Use scikit-learn's NearestNeighbors
# We ask for k+1 neighbors because the point itself is always the closest (distance 0)
nbrs = NearestNeighbors(n_neighbors=k + 1, algorithm='ball_tree').fit(features)
distances, indices = nbrs.kneighbors(features)

In [9]:
# --- 3. Define Edges ---
# Create an edge list based on k-NN
# indices[i, 0] is always node i itself, so we start from indices[i, 1]
edge_list = []
for i in range(len(features)):
    for j_idx in range(1, k + 1): # Iterate through the k nearest neighbors (excluding self)
        neighbor_index = indices[i, j_idx]
        # Add edge (i, neighbor_index) - ensure order doesn't matter for undirected graph
        # We can add edges in both directions initially and NetworkX handles duplicates
        edge_list.append(tuple(sorted((i, neighbor_index))))


# Remove duplicate edges by converting to a set
unique_edges = set(edge_list)

# --- 4. Build Graph ---
# Create an empty graph
G = nx.Graph()

# Add nodes
G.add_nodes_from(nodes)

# Add edges
G.add_edges_from(unique_edges)

# Optional: Add features/labels as node attributes
# Convert labels to string type if they aren't already, for compatibility
attributes = df_sar.to_dict('index') # Get attributes for each node (row index)
nx.set_node_attributes(G, attributes)

In [10]:
print(f"Graph created with k={k}")
print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")

Graph created with k=3
Number of nodes: 40
Number of edges: 81


In [11]:
nx.draw(G, with_labels=True, node_size=50, font_size=8)
plt.savefig("graph_dataset.png", dpi=600)

# PCC

In [ ]:
p_det = 0.6  # Probability of deterministic movement
iterations = 100 # Number of iterations
M = 3
G_sim, parts = run_simulation(G, M, iterations)
visualize_communities(G_sim)

In [49]:
check_average_node_potential(G_sim)


0.9392307772748885

In [36]:
get_dict_nodes_owner(G_sim)

{0: 2,
 1: 2,
 2: 0,
 3: 2,
 4: 0,
 5: 0,
 6: 2,
 7: 2,
 8: None,
 9: 1,
 10: 2,
 11: 2,
 12: 0,
 13: 0,
 14: 0,
 15: 0,
 16: 2,
 17: 2,
 18: 2,
 19: 2,
 20: 1,
 21: 1,
 22: 1,
 23: 2,
 24: 1,
 25: 1,
 26: 1,
 27: 1,
 28: 1,
 29: 2,
 30: 1,
 31: 1,
 32: 1,
 33: 2,
 34: 1,
 35: 2,
 36: 2,
 37: 1,
 38: 1,
 39: 1}

In [ ]:
df_dict_clusters = pd.DataFrame.from_dict(get_dict_nodes_owner(G_sim), orient='index',columns=['cluster'])
df_sar['cluster'] = df_dict_clusters['cluster']

for cluster in df_sar['cluster'].unique():
    arr_labels = df_sar.loc[df_sar['cluster'] == cluster, 'observed_label'].values
    if check_cluster_label(arr_labels):
        df_sar.loc[df_sar['cluster'] == cluster, 'cluster_label'] = 1
    else:
        df_sar.loc[df_sar['cluster'] == cluster, 'cluster_label'] = 0

df_sar.groupby('cluster')['observed_label'].apply(list).to_dict()

{0: [0, 0, 0, 0, 0, 0, 0],
 1: [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 2: [1, 0, 1, 0, 1, 1, 0, 0, 0]}